In [1]:
# base libraries
import json
import getpass

# industry-standard libraries
import numpy as np
import pandas as pd
import seaborn as sns

# snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as spf
from snowflake.snowpark import types as spt
from snowflake.snowpark import Window

# local environment
# pip install 'snowflake-connector-python[pandas]'
# conda install snowflake-snowpark-python pandas
# packages on snowflake
# select * from information_schema.packages where package_name = 'pandas'

In [2]:
if True:
    import snowflake
    print('numpy: ', np.__version__)
    print('pandas: ', pd.__version__)
    print('snowflake.snowpark: ', snowflake.snowpark.__version__)

numpy:  1.24.4
pandas:  2.0.3
snowflake.snowpark:  1.9.0


In [3]:
with open('_credentials.json') as f:
    connection_parameters = json.load(f)

connection_parameters['password'] = getpass.getpass('enter password: ')
session = Session.builder.configs(connection_parameters).create()

enter password:  ········


In [4]:
today = session.sql('''select replace(current_date::varchar, '-', '')''').collect()[0][0]
today

'20231024'

In [5]:
session.sql('create or replace database d{}'.format(today)).show()

--------------------------------------------
|"status"                                  |
--------------------------------------------
|Database D20231024 successfully created.  |
--------------------------------------------



In [6]:
# generate numpy 4D array
foo = np.random.random((5, 4, 3, 2))
foo

array([[[[0.86992939, 0.62902364],
         [0.85546488, 0.88138912],
         [0.01443393, 0.14447652]],

        [[0.64583976, 0.89514001],
         [0.19531514, 0.82296091],
         [0.47659617, 0.45519418]],

        [[0.76166311, 0.06683585],
         [0.98017252, 0.22769728],
         [0.47388212, 0.94741482]],

        [[0.11021056, 0.94607699],
         [0.16912818, 0.39878141],
         [0.76858257, 0.45214995]]],


       [[[0.08601604, 0.10365781],
         [0.70297941, 0.96678093],
         [0.06780132, 0.89525569]],

        [[0.29227512, 0.31724986],
         [0.28408278, 0.07046549],
         [0.3711202 , 0.17508809]],

        [[0.48149192, 0.3646056 ],
         [0.08151351, 0.02185468],
         [0.49759999, 0.24576044]],

        [[0.542856  , 0.29308041],
         [0.56100288, 0.33193885],
         [0.73645595, 0.03425081]]],


       [[[0.16878323, 0.11370279],
         [0.59554529, 0.92611182],
         [0.27520516, 0.6162064 ]],

        [[0.97598095, 0.18754024]

In [7]:
# put the numpy array into pandas dataframe
bar = pd.DataFrame(data=[[41, foo]], columns=('id', 'm'))
bar

,id,m
0,41,"[[[[0.86992939 0.62902364], [0.85546488 0.8813..."


In [8]:
# try to upload -> fail
df = session.create_dataframe(bar)

ArrowInvalid: ('Can only convert 1-dimensional array values', 'Conversion failed for column m with type object')

In [9]:
# convert numpy array to array-of-arrays
bar = pd.DataFrame(data=[[41, foo.tolist()]], columns=('id', 'm'))
bar

,id,m
0,41,"[[[[0.8699293876607219, 0.629023637362339], [0..."


In [10]:
# upload
df = session.create_dataframe(bar)
df.schema

StructType([StructField('"id"', LongType(), nullable=True), StructField('"m"', VariantType(), nullable=True)])

In [11]:
# query
df.show()

-----------------------------------------
|"id"  |"m"                             |
-----------------------------------------
|41    |[                               |
|      |  [                             |
|      |    [                           |
|      |      [                         |
|      |        8.699293876607219e-01,  |
|      |        6.290236373623390e-01   |
|      |      ],                        |
|      |      [                         |
|      |        8.554648760767676e-01,  |
|      |        8.813891165034791e-01   |
|      |      ],                        |
|      |      [                         |
|      |        1.443392852716008e-02,  |
|      |        1.444765200605064e-01   |
|      |      ]                         |
|      |    ],                          |
|      |    [                           |
|      |      [                         |
|      |        6.458397648730202e-01,  |
|      |        8.951400119272716e-01   |
|      |      ],                  